In [1]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from clickhouse_driver import Client
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import os
from lxml import html
import re
import datetime

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
sec = 'FXES'
driver.get(f'https://www.tinkoff.ru/invest/stocks/{sec}/pulse/')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/aleksandraromanenko/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [3]:
page_length = driver.execute_script("return document.body.scrollHeight")
while page_length < 9898 + 1:
    try:
        driver.execute_script(f"window.scrollTo(0, {page_length - 1000});")
        page_length = driver.execute_script("return document.body.scrollHeight")
    except:
        pass

In [4]:
source_data = driver.page_source
soup = bs(source_data, 'lxml')

In [5]:
posts = soup.find_all('div', {'class':'PulsePostCollapsed__text_3nQ1n PulsePostCollapsed__fourOrFiveLines_qVFqa'})
logins = soup.find_all('div', {'class':'PulsePostAuthor__nicknameLink_1p4Zx'})
likes = soup.find_all('div', {'class':'PulsePostBody__likes_130ao'})
times = soup.find_all('div', {'class':'PulsePostAuthor__inserted_Ctbka'})
        
logins = [login.text for login in logins]
posts = [post.text for post in posts]
likes = [int(like.text.split()[0]) for like in likes]
times = [time.text for time in times]

df_posts = pd.DataFrame()
df_posts['login'] = logins
df_posts['post'] = posts
df_posts['likes'] = likes
df_posts['time'] = times


In [6]:
def time(tim):
    if 'мин' in tim:
        ex = datetime.datetime.now() - datetime.timedelta(minutes=int(tim.split(' ')[0]))
        return ex.strftime(format='%d-%m-%Y %H:%m')
    elif 'час' in tim:
        if tim.split()[0] == 'час':
            ex = datetime.datetime.now() - datetime.timedelta(hours=1)
            return ex.strftime(format='%d-%m-%Y %H:%m')
        else:
            ex = datetime.datetime.now() - datetime.timedelta(hours=int(tim.split(' ')[0]))
            return ex.strftime(format='%d-%m-%Y %H:%m')
    elif 'вчера' in tim.lower():
        yester = datetime.datetime.now() - datetime.timedelta(days=1)
        return yester.strftime(format='%d-%m-%Y ') + tim.split(' ')[-1]

In [7]:
df_posts.time = df_posts.time.apply(lambda x: time(x))

In [8]:
df_posts['time'] = pd.to_datetime(df_posts['time'], format='%d-%m-%Y %H:%M')

In [9]:
df_posts.head(3)

,login,post,likes,time
0,morcvec,"Посмотрел новый ETF от Финекса, $FXES, меня оч...",9,2021-07-15 15:07:00
1,BonSai_108,Добавляю в мой долгосрочный портфель (ИИС- дру...,4,2021-07-15 15:07:00
2,vova.bless,$FXES 🕹 FinEx запустил новый фонд на видеоигры...,1,2021-07-15 15:07:00


In [83]:
from tqdm import tqdm 
logins = df_posts.login.unique()
subscribers = []
subscriptions = []
cap = [] 
deals = []
percents = []
colors = []
for login in tqdm(logins):
    url = f'https://www.tinkoff.ru/invest/social/profile/{login}/'
    response = requests.get(url)
    tree = bs(response.content, 'html.parser')
    subs = list(map(lambda x: x.split()[0], tree.find('div', {'class':'ProfileHeader__panel_3zzI4'}).text.split('•')))
    subscribers.append(subs[0])
    subscriptions.append(subs[1])
    
    summa = tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[0].text.replace('\xa0', ' ')
    amount = ''.join(summa.split( )[1:-1])
    cap.append(summa.split( )[0] + ' ' + amount + ' ' + summa.split( )[-1])
    
    deals.append(int(tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[1].text.split()[0]))
    
    
    percent = float(tree.find_all('div', {'class':'ProfileHeader__itemValue_2mH8M'})[2].text.split(
    )[0].split('%')[0])
    color = str(tree.find_all('span', {'class':'Icon-module__container_2fX7e'})[0]).split(
        'color:')[1].split(';')[0]
    
    if color == '#22a053':
        percents.append(percent)
    elif color == '#dd5656':
        percents.append(percent * -1)
    else:
        percents.append('-')
    
    

100%|██████████| 28/28 [02:11<00:00,  4.70s/it]


In [84]:
df_logs = pd.DataFrame()
df_logs['login'] = logins
df_logs['subscribers'] = subscribers
df_logs['subscriptions'] = subscriptions
df_logs['Объем портфеля'] = cap
df_logs['Сделок за 30 дней'] = deals
df_logs['За год'] = percents

In [88]:
df = df_posts.merge(df_logs, how='left', on='login')

In [104]:
df['Объем портфеля'].unique()

array(['до 3000000 ₽', 'до 500000 ₽', 'до 50000 ₽', 'до 10000 ₽',
       'от 3000000 ₽', 'до 100000 ₽', 'до 1000000 ₽'], dtype=object)

In [91]:
df.post.str.split(expand=True).stack().value_counts()

и             73
в             71
-             48
на            45
не            30
              ..
Roblox         1
рисков:        1
Среди          1
видеоигр:      1
задержкой.     1
Length: 1632, dtype: int64

In [99]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import unicodedata
import collections

nltk.download("stopwords")
nltk.download('punkt')
russian_stopwords = stopwords.words("russian")
append_stopword = ['это', 'FXES', 'https', 'fxes', 'finex', 'финекс']
russian_stopwords.extend(append_stopword)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandraromanenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandraromanenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
mystem = Mystem() 

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [102]:
df['lemm_post'] = df.post.apply(preprocess_text)

In [107]:
df.head()

,login,post,likes,time,subscribers,subscriptions,Объем портфеля,Сделок за 30 дней,За год,lemm_post
0,morcvec,"Посмотрел новый ETF от Финекса, $FXES, меня оч...",9,2021-07-15 15:07:00,3874,3,до 3000000 ₽,21,17.44,посмотреть новый etf очень смущать доля 7 5 4....
1,BonSai_108,Добавляю в мой долгосрочный портфель (ИИС- дру...,4,2021-07-15 15:07:00,2952,19,до 500000 ₽,184,19.57,добавлять долгосрочный портфель иис брокер нов...
2,vova.bless,$FXES 🕹 FinEx запустил новый фонд на видеоигры...,1,2021-07-15 15:07:00,41,19,до 500000 ₽,14,28.89,🕹 запускать новый фонд видеоигра киберспорт «...
3,Serg0787,$FXES как купить в долларах этот фонд?,1,2021-07-15 14:07:00,7,13,до 3000000 ₽,119,23.65,купить доллар фонд
4,ProInvestNews,#etf #новыйфонд #finex #разбор #new #игры #gam...,0,2021-07-15 14:07:00,107,27,до 500000 ₽,158,22.01,etf новыйфонд разбор new игра games 14 июль на...


In [155]:
!pip install dostoevsky

     |████████████████████████████████| 68 kB 1.2 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python3.7 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/10/_z2ffwmd113dsbkyc8cpg8mc0000gn/T/pip-install-hzncnuhk/fasttext/setup.py'"'"'; __file__='"'"'/private/var/folders/10/_z2ffwmd113dsbkyc8cpg8mc0000gn/T/pip-install-hzncnuhk/fasttext/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/10/_z2ffwmd113dsbkyc8cpg8mc0000gn/T/pip-wheel-2ub4btwg
       cwd: /private/var/folders/10/_z2ffwmd113dsbkyc8cpg8mc0000gn/T/pip-install-hzncnuhk/fasttext/
  Complete output (58 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.7
  creating build/lib.macosx-10.9-x86_64-3.7/fasttext
  copying pyt

In [156]:
!python -m dostoevsky download fasttext-social-network-model

/opt/anaconda3/bin/python: No module named dostoevsky
